In [ ]:
!pip install pyspark


In [ ]:
from pyspark.sql import SparkSession


SPARK_ENDPOINT = "local[*]"
sparkSession = SparkSession.builder \
    .master(SPARK_ENDPOINT) \
    .getOrCreate()
sparkContext = sparkSession.sparkContext
sparkSession


21/11/16 20:24:59 WARN Utils: Your hostname, bigdata resolves to a loopback address: 127.0.1.1; using 192.168.163.133 instead (on interface ens33)
21/11/16 20:24:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/16 20:25:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/16 20:25:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/11/16 20:25:03 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/11/16 20:25:03 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
21/11/16 20:25:03 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, StringType, StructField, StructType


MOVIES_INPUT_PATH = "movies.csv"
moviesDataFrame = sparkSession.read.csv(MOVIES_INPUT_PATH, header = True) \
    .withColumn("MovieId", col("MovieId").cast(IntegerType())) \
    .withColumn("Title", col("Title").cast(StringType())) \
    .withColumn("Genres", col("Genres").cast(StringType()))
moviesDataFrame.show()
moviesRdd = moviesDataFrame.rdd.map(lambda x: (int(x[0]), str(x[1]), str(x[2])))

+-------+--------------------+--------------------+
|MovieId|               Title|              Genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType, IntegerType


INPUT_PATH = "filtered150movies.csv"
dataFrame = sparkSession.read.csv(INPUT_PATH, header = True) \
    .withColumn("UserId", col("UserId").cast(IntegerType())) \
    .withColumn("MovieId", col("MovieId").cast(IntegerType())) \
    .withColumn("Rating", col("Rating").cast(FloatType()))
dataFrame.show()
rdd = dataFrame.rdd.map(lambda x: (int(x[0]), int(x[1]), float(x[2])))


+------+-------+------+
|UserId|MovieId|Rating|
+------+-------+------+
|     1|      1|   1.5|
|     5|      1|   1.5|
|     7|      1|   2.0|
|    15|      1|  -0.5|
|    17|      1|   2.0|
|    18|      1|   1.0|
|    19|      1|   1.5|
|    21|      1|   1.0|
|    27|      1|   0.5|
|    31|      1|   2.5|
|    32|      1|   0.5|
|    33|      1|   0.5|
|    40|      1|   2.5|
|    43|      1|   2.5|
|    44|      1|   0.5|
|    45|      1|   1.5|
|    46|      1|   2.5|
|    50|      1|   0.5|
|    54|      1|   0.5|
|    57|      1|   2.5|
+------+-------+------+
only showing top 20 rows



In [ ]:
cartesianRdd = rdd.cartesian(rdd) \
    .map(lambda x: (
        (x[0][0], x[0][1], x[0][2]), 
        (x[1][0], x[1][1], x[1][2])
    )) \
    .filter(lambda x: x[0][0] == x[1][0]) \
    .filter(lambda x: x[0][1] < x[1][1])
cartesianDataFrame = sparkSession.createDataFrame(cartesianRdd, ["(u, p1, r1)", "(u, p2, r2)"])
cartesianDataFrame.show()


+-----------+-------------+
|(u, p1, r1)|  (u, p2, r2)|
+-----------+-------------+
|{1, 1, 1.5}|  {1, 3, 1.5}|
|{1, 1, 1.5}|  {1, 6, 1.5}|
|{1, 1, 1.5}| {1, 47, 2.5}|
|{1, 1, 1.5}| {1, 50, 2.5}|
|{1, 1, 1.5}| {1, 70, 0.5}|
|{1, 1, 1.5}|{1, 110, 1.5}|
|{1, 1, 1.5}|{1, 151, 2.5}|
|{1, 1, 1.5}|{1, 163, 2.5}|
|{1, 1, 1.5}|{1, 216, 2.5}|
|{1, 1, 1.5}|{1, 223, 0.5}|
|{1, 1, 1.5}|{1, 231, 2.5}|
|{1, 1, 1.5}|{1, 235, 1.5}|
|{1, 1, 1.5}|{1, 260, 2.5}|
|{1, 1, 1.5}|{1, 296, 0.5}|
|{1, 1, 1.5}|{1, 316, 0.5}|
|{1, 1, 1.5}|{1, 333, 2.5}|
|{1, 1, 1.5}|{1, 349, 1.5}|
|{1, 1, 1.5}|{1, 356, 1.5}|
|{1, 1, 1.5}|{1, 362, 2.5}|
|{1, 1, 1.5}|{1, 367, 1.5}|
+-----------+-------------+
only showing top 20 rows



Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 663, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [ ]:
from math import pow


userProductRatingsRdd = cartesianRdd.map(lambda x: (
    (x[0][1], x[1][1]), 
    (x[0][2] * x[1][2], pow(x[0][2], 2), pow(x[1][2], 2))
))
userProductRatingsDataFrame = sparkSession.createDataFrame(userProductRatingsRdd, ["(p1, p2)", "(r1 * r2, r1^2, r2^2)"])
userProductRatingsDataFrame.show()

+--------+---------------------+
|(p1, p2)|(r1 * r2, r1^2, r2^2)|
+--------+---------------------+
|  {1, 3}|   {2.25, 2.25, 2.25}|
|  {1, 6}|   {2.25, 2.25, 2.25}|
| {1, 47}|   {3.75, 2.25, 6.25}|
| {1, 50}|   {3.75, 2.25, 6.25}|
| {1, 70}|   {0.75, 2.25, 0.25}|
|{1, 110}|   {2.25, 2.25, 2.25}|
|{1, 151}|   {3.75, 2.25, 6.25}|
|{1, 163}|   {3.75, 2.25, 6.25}|
|{1, 216}|   {3.75, 2.25, 6.25}|
|{1, 223}|   {0.75, 2.25, 0.25}|
|{1, 231}|   {3.75, 2.25, 6.25}|
|{1, 235}|   {2.25, 2.25, 2.25}|
|{1, 260}|   {3.75, 2.25, 6.25}|
|{1, 296}|   {0.75, 2.25, 0.25}|
|{1, 316}|   {0.75, 2.25, 0.25}|
|{1, 333}|   {3.75, 2.25, 6.25}|
|{1, 349}|   {2.25, 2.25, 2.25}|
|{1, 356}|   {2.25, 2.25, 2.25}|
|{1, 362}|   {3.75, 2.25, 6.25}|
|{1, 367}|   {2.25, 2.25, 2.25}|
+--------+---------------------+
only showing top 20 rows



In [ ]:
step2Rdd = userProductRatingsRdd.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2]))
step2DataFrame = sparkSession.createDataFrame(step2Rdd, ["(p1, p2)", "(pra1,2 + prb1,2, ra1^2 + rb1^2, ra2^2 + rb2^2)"])
step2DataFrame.show()

+--------+-----------------------------------------------+
|(p1, p2)|(pra1,2 + prb1,2, ra1^2 + rb1^2, ra2^2 + rb2^2)|
+--------+-----------------------------------------------+
|  {1, 3}|                            {48.0, 88.25, 51.5}|
|  {1, 6}|                           {119.0, 150.25, 1...|
| {1, 47}|                           {190.0, 243.5, 29...|
| {1, 50}|                           {240.25, 255.0, 3...|
| {1, 70}|                           {31.75, 68.75, 61...|
|{1, 110}|                           {227.75, 287.25, ...|
|{1, 151}|                           {37.25, 51.75, 42.0}|
|{1, 163}|                            {66.5, 80.0, 88.25}|
|{1, 216}|                           {27.75, 100.75, 6...|
|{1, 223}|                           {98.0, 139.5, 153.5}|
|{1, 231}|                           {42.25, 201.75, 1...|
|{1, 235}|                           {67.5, 101.25, 86...|
|{1, 260}|                           {327.25, 349.5, 5...|
|{1, 296}|                           {326.25, 375.75, ..

In [ ]:
from math import sqrt


cosineRdd = step2Rdd.map(lambda x: (
    x[0][0], 
    x[0][1], 
    x[1][0] / ( sqrt(x[1][1]) * sqrt(x[1][2]) )
))
cosineDataFrame = sparkSession.createDataFrame(cosineRdd, ["Movie1Id", "Movie2Id", "CosineDistance"])
cosineDataFrame.show()

+--------+--------+-------------------+
|Movie1Id|Movie2Id|     CosineDistance|
+--------+--------+-------------------+
|       1|       3| 0.7120004462365306|
|       1|       6| 0.7663059588674602|
|       1|      47| 0.7134624997997026|
|       1|      50| 0.8064987231508762|
|       1|      70|0.48729156902469273|
|       1|     110| 0.7012089727414182|
|       1|     151|  0.798999403009002|
|       1|     163| 0.7914423667141078|
|       1|     216|0.34970388040486955|
|       1|     223| 0.6697069130757897|
|       1|     231|0.27013388711672426|
|       1|     235| 0.7223151185146152|
|       1|     260| 0.7628811383332508|
|       1|     296| 0.7324602560370937|
|       1|     316|0.48769118113363935|
|       1|     333| 0.6741896857847387|
|       1|     349| 0.7253894119809242|
|       1|     356| 0.7946310146221903|
|       1|     362| 0.5916108640359643|
|       1|     367| 0.4724236978477104|
+--------+--------+-------------------+
only showing top 20 rows



In [ ]:
movies1DataFrame = sparkSession.createDataFrame(moviesRdd, ["MovieId1", "Title1", "Genres1"])
movies2DataFrame = sparkSession.createDataFrame(moviesRdd, ["MovieId2", "Title2", "Genres2"])
cosineWithTitlesRdd = cosineDataFrame \
    .join(movies1DataFrame, cosineDataFrame.Movie1Id == movies1DataFrame.MovieId1, "inner") \
    .join(movies2DataFrame, cosineDataFrame.Movie2Id == movies2DataFrame.MovieId2, "inner") \
    .rdd \
    .map(lambda x: (x[0], x[4], x[1], x[7], x[2])) \
    .sortBy(lambda x: -x[4])
cosineWithTitlesDataFrame = sparkSession.createDataFrame(cosineWithTitlesRdd, ["MovieId1", "MovieTitle1", "MovieId2", "MovieTitle2", "CosineDistance"])
cosineWithTitlesDataFrame.show(10)

+--------+--------------------+--------+--------------------+------------------+
|MovieId1|         MovieTitle1|MovieId2|         MovieTitle2|    CosineDistance|
+--------+--------------------+--------+--------------------+------------------+
|     151|      Rob Roy (1995)|     441|Dazed and Confuse...|               1.0|
|     260|Star Wars: Episod...|    1196|Star Wars: Episod...|0.9560939845339547|
|    1090|      Platoon (1986)|    1213|   Goodfellas (1990)|0.9504490201227582|
|     661|James and the Gia...|     923| Citizen Kane (1941)|0.9476070829586856|
|     151|      Rob Roy (1995)|    1080|Monty Python's Li...|0.9472110029417574|
|     923| Citizen Kane (1941)|    1587|Conan the Barbari...|0.9435183924675248|
|     151|      Rob Roy (1995)|    2329|American History ...|0.9434864536402258|
|    1198|Raiders of the Lo...|    1291|Indiana Jones and...| 0.942606433250396|
|     362|Jungle Book, The ...|    1587|Conan the Barbari...|0.9417419115948374|
|     923| Citizen Kane (194